# Crear la clase  Target



In [24]:
import duckdb
import pandas as pd
import os
import time


In [25]:
# Rutas absolutas a los datasets crudos comprimidos
DATASET_PATH_COMP_2 = "/home/sanmartinofacundo/datasets/competencia_02_crudo.csv.gz"

In [26]:
conn = duckdb.connect(database=":memory:")

# 🔌 Habilitar soporte HTTP/HTTPS
conn.execute("INSTALL httpfs;")
conn.execute("LOAD httpfs;")

# (Opcional pero recomendable)
conn.execute("SET enable_http_metadata_cache=true;")
conn.execute("SET enable_object_cache=true;")

# Ruta de competencia_02 local (como ya tenías)
DATASET_PATH_COMP_2 = "/home/sanmartinofacundo/datasets/competencia_02_crudo.csv.gz"

start = time.time()

# 1) Crear tabla base con competencia_02
conn.execute(f"""
    CREATE OR REPLACE TABLE competencia_crudo_completo AS
    SELECT * FROM read_csv_auto('{DATASET_PATH_COMP_2}');
""")
print("🟢 Tabla base (competencia_02) creada.")

# 2) Insertar competencia_03 directamente desde la URL pública
url_comp_3 = "https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_03_crudo.csv.gz"

conn.execute(f"""
    INSERT INTO competencia_crudo_completo
    SELECT * FROM read_csv_auto('{url_comp_3}');
""")
print("🟢 Datos de competencia_03 agregados desde la URL.")

end = time.time()
print(f"⏱ Todo listo en {end - start:.2f} segundos")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🟢 Tabla base (competencia_02) creada.


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

🟢 Datos de competencia_03 agregados desde la URL.
⏱ Todo listo en 58.11 segundos


In [27]:
start = time.time()

conn.execute("""
    CREATE OR REPLACE TABLE competencia_03 AS
    WITH base AS (
        SELECT
            *,
            -- ✅ versión correcta del periodo0 (entero)
            CAST(FLOOR(foto_mes / 100) AS BIGINT) * 12
              + (foto_mes % 100) AS periodo0
        FROM competencia_crudo_completo
    ),
    datos AS (
        SELECT
            *,
            LEAD(periodo0, 1) OVER (
                PARTITION BY numero_de_cliente
                ORDER BY periodo0
            ) AS periodo1,
            LEAD(periodo0, 2) OVER (
                PARTITION BY numero_de_cliente
                ORDER BY periodo0
            ) AS periodo2
        FROM base
    ),
    maximos AS (
        SELECT
            MAX(periodo0) AS periodo_ultimo,
            MAX(periodo0) - 1 AS periodo_anteultimo
        FROM base
    ),
    etiquetado AS (
        SELECT
            d.*,
            CASE
                -- BAJA+2
                WHEN d.periodo0 <  m.periodo_anteultimo
                     AND d.periodo0 + 1 = d.periodo1
                     AND (d.periodo2 IS NULL OR d.periodo0 + 2 < d.periodo2)
                THEN 'BAJA+2'

                -- BAJA+1
                WHEN d.periodo0 < m.periodo_ultimo
                     AND (d.periodo1 IS NULL OR d.periodo0 + 1 < d.periodo1)
                THEN 'BAJA+1'

                -- CONTINUA
                WHEN d.periodo0 < m.periodo_anteultimo
                THEN 'CONTINUA'

                ELSE NULL
            END AS clase_ternaria
        FROM datos d
        CROSS JOIN maximos m
    )
    SELECT
        * EXCLUDE (periodo0, periodo1, periodo2)
    FROM etiquetado;
""")

end = time.time()
print(f"✅ Tabla 'competencia_03' creada en memoria en {end - start:.2f} segundos")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Tabla 'competencia_03' creada en memoria en 12.11 segundos


In [28]:
# Ejecutar la consulta SQL en DuckDB y obtener un DataFrame
df_resumen = conn.execute("""
    SELECT 
        foto_mes, 
        clase_ternaria, 
        COUNT(*) AS cantidad
    FROM competencia_03
    GROUP BY foto_mes, clase_ternaria
    ORDER BY foto_mes, clase_ternaria
""").df()

# Mostrar resultado
df_resumen.head(15)


,foto_mes,clase_ternaria,cantidad
0,201901,BAJA+1,645
1,201901,BAJA+2,729
2,201901,CONTINUA,122899
3,201902,BAJA+1,733
4,201902,BAJA+2,707
5,201902,CONTINUA,123961
6,201903,BAJA+1,708
7,201903,BAJA+2,751
8,201903,CONTINUA,124508
9,201904,BAJA+1,756


In [29]:
# Ejecutar la consulta PIVOT en DuckDB y obtener un DataFrame
df_pivot = conn.execute("""
    PIVOT competencia_03
    ON clase_ternaria
    USING count(numero_de_cliente)
    GROUP BY foto_mes
""").df()

# Mostrar resultado
df_pivot.head(36)

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,645,729,122899
1,201902,733,707,123961
2,201903,708,751,124508
3,201904,756,514,125268
4,201905,517,692,125993
5,201906,696,608,127430
6,201907,611,680,128977
7,201908,683,577,130883
8,201909,581,582,132594
9,201910,594,618,134798


In [30]:
# Guardar en bucket de GCP (b1)
dataset_path_bucket = "/home/sanmartinofacundo/buckets/b1/competencia_03.csv.gz"

os.makedirs(os.path.dirname(dataset_path_bucket), exist_ok=True)

conn.execute(f"""
    COPY competencia_03 
    TO '{dataset_path_bucket}' 
    (FORMAT CSV, HEADER, COMPRESSION GZIP);
""")

print("✅ Guardado directamente en el bucket montado b1")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

✅ Guardado directamente en el bucket montado b1
